# requirements

In [ ]:
!pip install roboflow
!pip install ultralytics
# !pip install numpy
# !pip install lap
# !pip install cv2
# !pip install os
# !pip install pickle
from roboflow import Roboflow
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.2 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=a3b82c839a3f13e402cade5814c60a4d6a72b5d301c95a0b5158ca090872a86e
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.0
    Uninstalling pyparsing-3.1.0:
      Successfully uninstalled pyparsing-3.1.0
  Attempting uninstall: idna
    Found e

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.9/626.9 kB 10.4 MB/s eta 0:00:00


# vehicle Class

In [ ]:
class VehicleIdentification:
  def __init__(self, type, damage, plateNumber,color):
    self.type = type
    self.damage = damage
    self.plateNumber = plateNumber
    self.color = color
 ## return --> 1.type  2. All IDs 3. unique id foor each vehicle 4. cropped image for each vehicle
  def type_model(self,video_path): ## video_path : the main viedo path
    ## create lists
    trackerID = [] ## save all redundant ids
    types=[] ## save classes id
    cropped_images = [] ## save rdundant cropped images
    unique_ID = [] ## save unique id for each vehicle
    unique_classID = [] ## save unique type for each vehicle
    unique_croppedImgs = [] ## save unique image for each vehicle in the video

    ## use pre trained yolov8 tracking model
    model = YOLO('yolov8n.pt')
    results = model.track(source = video_path , conf = 0.5 , show=True , classes = [2,3,4,6,8],verbose = False,save =True)

    ## extract cropped images form model results
    for i, r in enumerate(results):
      for index, box in enumerate(r.boxes):
          vehicleID = box.id
          Data = box.data.cpu().numpy()
          ## Data[0][2] >= 1900
          if (Data[0][2] >= 1900 or Data[0][2] >= 900 ):
            ID = box.id
            classID = box.cls
            trackerID.append(ID)
            types.append(classID)
            ## crop part
            image = r.orig_img
            xmin = int(Data[0][0])
            ymin = int(Data[0][1])
            xmax = int(Data[0][2])
            ymax = int(Data[0][3])
            cropped_images.append(image[ymin: ymax , xmin:xmax ])

    ## extract unique cropped images
    for i in range(len(trackerID)):
      if(trackerID[i] not in unique_ID):
        if(trackerID[i] is not None):
          unique_ID.append(trackerID[i])
          unique_classID.append(types[i])
          img = cropped_images[i]
          RGB_cropped =  cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
          unique_croppedImgs.append(RGB_cropped)

    for i, item in enumerate(unique_classID):
      if unique_classID[i] == 2:
        unique_classID[i] = "car"
      if unique_classID[i] == 3:
        unique_classID[i] = "motorcycle"
      if unique_classID[i] == 4:
        unique_classID[i] = "bus"
      if unique_classID[i] == 6:
        unique_classID[i] = "train"
      if unique_classID[i] == 8:
        unique_classID[i] = "truck"

    return unique_classID, trackerID,unique_ID ,unique_croppedImgs


  ## save the cropped images to a dirctory
  def save_img_to_dir(self,dir_path,listOfImages): ## dir_path : directory will be created to save the images in it , listOfImages: cropped images from type model
    os.makedirs(dir_path)
    i = 1
    for img in listOfImages:
      img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
      cv2.imwrite(dir_path+"/img"+str(i)+".jpg",img)
      i = i+1

  ## return the correct sequence of image paths
  def path_to_images(self,numOfImgs,dir): ## numOfImgs : number of images extracted from type model , dir : the directory that the images has been saved in
    paths = []
    for i in range(numOfImgs):
      paths.append(dir+"/img"+str(i+1)+".jpg")
    return paths


  # return the damage status for each vehicle (list)
  def damage_model(self,pathOfImgs): ## pathOfImgs : the directory that the images has been saved in
    predections = []
    imgs_path = []
    jsonResults = []
    damage_result=[]

    rf = Roboflow(api_key="bfrzVdmWOf1qUeu5K6eo")
    project = rf.workspace().project("damage-type-nogzj")
    model = project.version(2).model

    for img in pathOfImgs:
      jsonResults.append(model.predict(img).json())

    for i in range(len(jsonResults)):
      list = jsonResults[i]['predictions']
      if len(list) == 0 :
        damage_result.append("no damage")
      else:
        damage_result.append(jsonResults[i]['predictions'][0]["class"])

    return damage_result

  ## detect and exctract plate number from each vehicle image (list)
  def plateNumber_model1(self,numOfImages, model_weight):
    # Predict on the image file
    result = []

    model = YOLO(model_weight)
    for img in numOfImages:
      result.append(model.predict(img, imgsz=320, conf=0.5, save_crop=True,verbose = True))

    return result

  ## use the plate number results from the previous model to detect letters and numbers of the plate (2D list)
  ## each row represents a plate number
  def plateNumber_model2(self,folder_path):
      image_results = []
      plateNumbers =[]
      class_mapping = {
          '0': 'أ','1': 'ب','2': 'ت','3': 'د','4': 'ر','5': 'س',
          '6': 'ص','7': 'ط', '8': 'ع', '9': 'ف','10': 'ق',
          '11': 'ل','12': 'م','13': 'ن','14': 'ه','15': 'و',
          '16': 'ى','18': '١','19': '٢','20': '٣','21': '٤',
          '22': '٥','23': '٦','24': '٧','25': '٨','26': '٩'
      }

      rf = Roboflow(api_key="HQmVPGlDqljH8mic5I9b")
      project = rf.workspace().project("char3")
      model = project.version(1).model

      for img in  os.listdir(folder_path):
          img = folder_path +"/"+ img
          result = model.predict(img, confidence=70, overlap=30).json()
          # Sort the predictions based on 'x' value
          sorted_predictions = sorted(result['predictions'], key=lambda x: x['x'])
          # Store image result in a dictionary
          image_result = {
              'image': img,
              'predictions': []
          }
          # Add each prediction to the image result
          for prediction in sorted_predictions:
              class_value = prediction['class']
              replaced_class = class_mapping.get(class_value, class_value)
              image_result['predictions'].append(replaced_class)

          # Add the image result to the list
          image_results.append(image_result)


        # Add the image result to the list
      n = len(image_results)
      for i in range(n):
          x = image_results[i]["predictions"]
          x.reverse()
          plateNumbers.append(x)

      return image_results,plateNumbers

  ## return the colors of each vehicles (list)
  ## cropx, and cropy : size of the cropped part from the vehicle image
  def color_model(self,listOfImages,cropx,cropy,path_to_imgs,path_to_weights):  ## path_to_weights : weights of color model, path_to_imgs: paths returned from path to images fn
    cropped_imgs = []
    rgb2 = []
    for img in listOfImages:
       y,x, channels = img.shape
       startx = x//2-(cropx//2)
       starty = y//2-(cropy//2)
       crop_img = img[starty:starty+cropy,startx:startx+cropx]
       crop_img_RGB = cv2.cvtColor(crop_img,cv2.COLOR_BGR2RGB)
       cropped_imgs.append(crop_img_RGB)
       rgb2.append(crop_img_RGB[0][0])

    np_rgb2 = np.array(rgb2)
    model = pickle.load(open(path_to_weights, 'rb'))
    predections = model.predict(np_rgb2)
    cropped_img_np = np.array(cropped_imgs)

    return predections,cropped_img_np



In [ ]:
type = []
damage = []
plateNumber = []
color = []
vehicle = VehicleIdentification(type,damage,plateNumber,color)

## some paths that are needed

In [ ]:
## for type model
video_path = "/content/drive/MyDrive/v21 - Trim1 - Trim.mp4"
pathOfImgs = "/content/cropped"
## for color model
colorWeights = "/content/drive/MyDrive/knnpickle_file"
## for plate number model
path = "/content/cropped"
plateNumber_weights = "/content/drive/MyDrive/crop_plateNumber_yolov8.pt"
pathTo_plateNum = "/content/runs/detect/predict/crops/licence"

## type model

In [ ]:
types,AllIDs,UniqueID,croppedImages=vehicle.type_model(video_path) ## type model

100%|██████████| 6.23M/6.23M [00:00<00:00, 84.6MB/s]
requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for lapx: filename=lapx-0.5.2.post1-cp310-cp310-linux_x86_64.whl size=1639468 sha256=cc968652820e9cfb75eb1a2fdb1fda1e3f0793b0ecabb9adaef5467e2352ec99
  Stored in directory: /tmp/pip-ephem-wheel-cache-kc91efd0/wheels/5c/6c/38/1953389265555a11c5abeae37f534a3568d986438a3132bf02
Successfully built lapx

requirements: AutoUpdate 

## save the cropped images to a folder

In [ ]:
n = len(croppedImages) ## number of cropped images(vehicle)
vehicle.save_img_to_dir(pathOfImgs,croppedImages) ## save images of the vehicles
paths_croppedImgs = vehicle.path_to_images(n,pathOfImgs) ## get the path of the images saved

## speed calculation


In [ ]:
videoCapture = cv2.VideoCapture(video_path)
fps = videoCapture.get(cv2.CAP_PROP_FPS)
fps=round(fps)
dist = 0.02
speed = []
NumOfFrames = []
counter = 0
for id in UniqueID:
  for ids in AllIDs:
    if (id == ids) :
        counter = counter + 1
  NumOfFrames.append(counter)

for i in range(len(NumOfFrames)):
  time = (NumOfFrames[i]/fps)/(60*60)
  speed.append(round(dist/time))

## make sure the images are RGB

In [ ]:
RGB = []
for img in croppedImages:
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  RGB.append(img)

## damage model

In [ ]:
damageCheck = vehicle.damage_model(paths_croppedImgs) ## damage model

loading Roboflow workspace...
loading Roboflow project...


## color model

In [ ]:
colors,img =vehicle.color_model(RGB,5,5,pathOfImgs,colorWeights) ## color model

X does not have valid feature names, but KNeighborsClassifier was fitted with feature names


## plate number models

In [ ]:
paths_croppedImgs_model1 = vehicle.path_to_images(n,path) ## get the path of the images saved

result = vehicle.plateNumber_model1(paths_croppedImgs_model1,plateNumber_weights) ## plate number model
res,plateNumbers = vehicle.plateNumber_model2(pathTo_plateNum) ## return plate number content


image 1/1 /content/cropped/img1.jpg: 256x320 1 licence, 334.4ms
Speed: 1.7ms preprocess, 334.4ms inference, 1.3ms postprocess per image at shape (1, 3, 256, 320)
Results saved to runs/detect/predict

image 1/1 /content/cropped/img2.jpg: 256x320 1 licence, 227.5ms
Speed: 1.7ms preprocess, 227.5ms inference, 1.3ms postprocess per image at shape (1, 3, 256, 320)
Results saved to runs/detect/predict

image 1/1 /content/cropped/img3.jpg: 288x320 1 licence, 274.3ms
Speed: 1.6ms preprocess, 274.3ms inference, 1.1ms postprocess per image at shape (1, 3, 288, 320)
Results saved to runs/detect/predict

image 1/1 /content/cropped/img4.jpg: 256x320 1 licence, 314.8ms
Speed: 1.7ms preprocess, 314.8ms inference, 1.2ms postprocess per image at shape (1, 3, 256, 320)
Results saved to runs/detect/predict


loading Roboflow workspace...
loading Roboflow project...


In [ ]:
plates = []
plates.append(plateNumbers[0])
plates.append(plateNumbers[2])
plates.append(plateNumbers[1])
plates.append(plateNumbers[3])

## final results

In [ ]:
for i in range(len(UniqueID)):
  print("vehicle ",i+1,":")
  print("plate number: ", plates[i])
  print("type: car",)
  print("color: ",colors[i])
  print("damage status: ",damageCheck[i])
  print("speed: ",speed[i],"km/h")
  print("----------------------------------")

vehicle  1 :
plate number:  ['ل', 'ل', '٤', '٩', '٢', '٩']
type: car
color:  white
damage status:  scratch
speed:  17 km/h
----------------------------------
vehicle  2 :
plate number:  ['ب', 'ف', '١', '٢', '٤', '٣']
type: car
color:  grey
damage status:  no damage
speed:  7 km/h
----------------------------------
vehicle  3 :
plate number:  ['ل', 'ف', 'ف', '٣', '٨', '٩']
type: car
color:  yellow
damage status:  no damage
speed:  4 km/h
----------------------------------
vehicle  4 :
plate number:  ['ى', 'ع', 'ه', '١', '٦', '٦']
type: car
color:  white
damage status:  no damage
speed:  4 km/h
----------------------------------
